In [28]:
import numpy as np
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [29]:
processed_text = pd.read_csv("processed_text2.csv")
# semantic_features = pd.read_csv("semantic_features.csv")
# readiblity_features = pd.read_csv("readiblity_features_1.csv")
syntax_features = pd.read_csv("syntax_features_1.csv")
# structure_features = pd.read_csv("structure_features_1.csv")
# sentiment_features = pd.read_csv("sentiment_features.csv")
# features1 = pd.concat((semantic_features,readiblity_features,syntax_features,structure_features,sentiment_features),axis=1)
features1 = syntax_features
features1 

,ADJ,NOUN,PUNCT,PROPN,PART,CCONJ,NUM,SYM,ADP,AUX,...,MONEY,NORP,ORDINAL,ORG,PERCENT,PERSON,PRODUCT,QUANTITY,TIME,WORK_OF_ART
0,0.102703,0.189189,0.178378,0.059459,0.043243,0.032432,0.021622,0.021622,0.059459,0.070270,...,1,0,0,2,0,3,0,0,0,0
1,0.082090,0.216418,0.082090,0.000000,0.037313,0.067164,0.000000,0.000000,0.082090,0.052239,...,0,0,0,0,0,0,0,0,0,0
2,0.077551,0.175510,0.134694,0.016327,0.024490,0.057143,0.004082,0.000000,0.048980,0.085714,...,0,0,0,0,0,0,0,0,0,0
3,0.102439,0.146341,0.107317,0.121951,0.019512,0.058537,0.029268,0.009756,0.073171,0.063415,...,2,0,0,2,0,4,0,0,0,0
4,0.085271,0.217054,0.104651,0.023256,0.019380,0.058140,0.003876,0.000000,0.073643,0.058140,...,0,2,0,0,0,1,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14849,0.064815,0.129630,0.148148,0.037037,0.023148,0.023148,0.000000,0.004630,0.050926,0.069444,...,0,0,0,0,0,2,0,0,0,0
14850,0.117647,0.176471,0.124183,0.019608,0.013072,0.058824,0.006536,0.000000,0.052288,0.124183,...,0,1,1,0,0,2,0,0,0,0
14851,0.087248,0.154362,0.073826,0.013423,0.026846,0.040268,0.013423,0.006711,0.120805,0.080537,...,0,3,0,0,0,1,0,0,0,0
14852,0.067961,0.208738,0.126214,0.082524,0.038835,0.024272,0.029126,0.000000,0.067961,0.072816,...,0,0,0,1,0,1,0,0,1,0


In [30]:
# Add the 'label' column from processed_text to the semantic_features dataframe
features1['label'] = processed_text['label']

# Now we have a new dataframe with all semantic features and the label
features1_detect = features1

# Display the first few rows of the new dataframe
features1_detect.shape
features1_detect

,ADJ,NOUN,PUNCT,PROPN,PART,CCONJ,NUM,SYM,ADP,AUX,...,NORP,ORDINAL,ORG,PERCENT,PERSON,PRODUCT,QUANTITY,TIME,WORK_OF_ART,label
0,0.102703,0.189189,0.178378,0.059459,0.043243,0.032432,0.021622,0.021622,0.059459,0.070270,...,0,0,2,0,3,0,0,0,0,0
1,0.082090,0.216418,0.082090,0.000000,0.037313,0.067164,0.000000,0.000000,0.082090,0.052239,...,0,0,0,0,0,0,0,0,0,0
2,0.077551,0.175510,0.134694,0.016327,0.024490,0.057143,0.004082,0.000000,0.048980,0.085714,...,0,0,0,0,0,0,0,0,0,0
3,0.102439,0.146341,0.107317,0.121951,0.019512,0.058537,0.029268,0.009756,0.073171,0.063415,...,0,0,2,0,4,0,0,0,0,0
4,0.085271,0.217054,0.104651,0.023256,0.019380,0.058140,0.003876,0.000000,0.073643,0.058140,...,2,0,0,0,1,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14849,0.064815,0.129630,0.148148,0.037037,0.023148,0.023148,0.000000,0.004630,0.050926,0.069444,...,0,0,0,0,2,0,0,0,0,1
14850,0.117647,0.176471,0.124183,0.019608,0.013072,0.058824,0.006536,0.000000,0.052288,0.124183,...,1,1,0,0,2,0,0,0,0,1
14851,0.087248,0.154362,0.073826,0.013423,0.026846,0.040268,0.013423,0.006711,0.120805,0.080537,...,3,0,0,0,1,0,0,0,0,1
14852,0.067961,0.208738,0.126214,0.082524,0.038835,0.024272,0.029126,0.000000,0.067961,0.072816,...,0,0,1,0,1,0,0,1,0,1


In [31]:
# Splitting the data into features (X) and target (y)
x = features1_detect.drop(columns=['label'])
y = features1_detect['label']

In [32]:
# 資料標準化
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(x)  # X 是原始的特徵資料

In [33]:
model = AdaBoostClassifier(random_state=42)

In [34]:
def specificity_score(y_true, y_pred):
    tn, fp, _, _ = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    return specificity

specificity_scorer = make_scorer(specificity_score, greater_is_better=True)
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'specificity': specificity_scorer
}

In [35]:
# 设置网格搜索的参数，注意这里的格式
param_grid = {
    'n_estimators':[50,100,200,300,400],
    'learning_rate':[0.001,0.01,0.05,0.1,0.3,0.5]
}

In [37]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, refit='f1', cv=10, n_jobs=-1)

# 假設X_normalized和y已經被定義且正確預處理
# 執行網格搜索
grid_result = grid.fit(X_normalized, y)



c:\Users\yichiiii17\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [38]:
grid_result.best_estimator_

AdaBoostClassifier(learning_rate=0.3, n_estimators=400, random_state=42)

In [39]:
print("最佳 Accuracy: %f 使用 %s" % (grid_result.cv_results_['mean_test_accuracy'][grid_result.best_index_], grid_result.best_params_))
print("最佳 Precision: %f 使用 %s" % (grid_result.cv_results_['mean_test_precision'][grid_result.best_index_], grid_result.best_params_))
print("最佳 Recall: %f 使用 %s" % (grid_result.cv_results_['mean_test_recall'][grid_result.best_index_], grid_result.best_params_))
print("最佳 Specificity: %f 使用 %s" % (grid_result.cv_results_['mean_test_specificity'][grid_result.best_index_], grid_result.best_params_))
print("最佳 F1: %f 使用 %s" % (grid_result.cv_results_['mean_test_f1'][grid_result.best_index_], grid_result.best_params_))


最佳 Accuracy: 0.732389 使用 {'learning_rate': 0.3, 'n_estimators': 400}
最佳 Precision: 0.725813 使用 {'learning_rate': 0.3, 'n_estimators': 400}
最佳 Recall: 0.721857 使用 {'learning_rate': 0.3, 'n_estimators': 400}
最佳 Specificity: 0.742494 使用 {'learning_rate': 0.3, 'n_estimators': 400}
最佳 F1: 0.720593 使用 {'learning_rate': 0.3, 'n_estimators': 400}


In [40]:
result={'Accuracy': grid_result.cv_results_['mean_test_accuracy'][grid_result.best_index_],
    'Precision': grid_result.cv_results_['mean_test_precision'][grid_result.best_index_],
    'Recall': grid_result.cv_results_['mean_test_recall'][grid_result.best_index_],
    'Specificity': grid_result.cv_results_['mean_test_specificity'][grid_result.best_index_],
    'F1': grid_result.cv_results_['mean_test_f1'][grid_result.best_index_],
    'Best_Params': [  {'learning_rate': 0.3, 'n_estimators': 400}]}
df_result = pd.DataFrame([result])
df_result.to_csv('syntax_ADA_1.csv',index=False)